# Welcome to Smart Donut Bot

In this notebook, we create a smart group of pairings that will allow you to make donut groups that meet new people! The initial few cells are to be run once, and then you can run specific cells on a weekly basis to generate new matches. Try not to restart the algorithm midway through running!

In [1]:
import pandas as pd
import numpy as np
import requests
from airtable import airtable
import fbchat
import time

In [ ]:
client = fbchat.Client('EMAIL', 'PASSWORD')

So as you can see, we've imported a few libraries above that will allow us to do some quick data manipulation! The cell below us is quite important as it is the place you are able to manipulate the donut bot's constraints. It is recommended not to change these constraints after you have started running the algorithm (for the first few weeks), but if you need do end up changing these constraints, the places with **bold text** are the cells you need to re-run.

In [ ]:
#CONSTANTS
N = 3 #Number of people in each group, if there's is excess, it will create groups of N+1
initial_weight = 1 #Starting weight for every member
same_joined_semester = 0.5 #If they joined the same semester, they will be weighted less likely to be matched. Make it 0 if you don't want it to be a factor
POPA = 0.01 #probability_of_paired_again 
table_name = "Table 2" #name of your airtable table in your Base

Also, upload a roster file for your student organization. The file should follow this format with the column names exactly matching the table. All columns except 'Name' and 'Class' are optional. 

| Name | Class       | Expected Graduation | Never Match       |
|:----:|-------------|||
| XYZ  | Spring 2020 |      |             |
| YSA  | Fall 2017   |      |             |
| ADE  | Spring 2019 |      |             |

In [ ]:
roster = pd.read_csv('') #INPUT FILEPATH TO ROSTER HERE
roster.head(5)

Now that we have imported the roster, we are going to generate a dictionary of dictionaries and set their initial weights. This will be the the probability that one person gets matched to another person. If you want to add more inital weights, you can do so at the comment. These next two cells should only ever be run *once*.

In [ ]:
memberNames = {}
n = roster['Name'].to_numpy()

In [ ]:
for name in n:
    memberNames[name] = {}
    year_joined = roster.loc[roster['Name'] == name, 'Class'].iloc[0]
    for name2 in n:
        if name != name2:
            memberNames[name][name2] = initial_weight
            year_joined_two = roster.loc[roster['Name'] == name2, 'Class'].iloc[0]
            if year_joined == year_joined_two:
                memberNames[name][name2] -= same_joined_semester
            #other weightings


Below is the code that matches members based on their weightings. This is what should be run every time you want to generate new **matches** and continue till the end of the notebook.

If you would like to change N (number of people in each group) copy and paste one of the blocks and change third to fourth, fifth etc. 

In [ ]:
groups = []
n_copy = n
while len(n) >= N:
    group = np.array([])
    first_rand = np.random.choice(n)
    group = np.append(group, first_rand)
    n = n[n != first_rand]
    
    probs = np.array([memberNames[first_rand][name] for name in n])
    probs = probs / probs.sum()
    second_rand = np.random.choice(n, 1, p=probs, replace=False)[0]
    n = n[n != second_rand]
    group = np.append(group,second_rand)

    #copy block
    probs = np.array([memberNames[first_rand][name] + memberNames[second_rand][name] for name in n])
    probs = probs / probs.sum()
    third_rand = np.random.choice(n, 1, p=probs, replace=False)
    n = n[n != third_rand]
    group = np.append(group,third_rand)
    #end copy
    
    groups.append(group)

for ind, remainder in enumerate(n):
    groups[0] = np.append(groups[0], remainder)

n = n_copy

If you want to change N, create Person_5, Person_6 etc...

In [ ]:
display_groups = pd.DataFrame(groups, columns=["Person_1", "Person_2", "Person_3", "Person_4"])
display_groups

The below code block will send messenger messages from your facebook account, enter your Facebook email and password below. None of this will be saved so your information is secure!

In [ ]:
#Implement Messenger Messaging
for index, row in display_groups.iterrows():
    if index >=7:
        messagList = []
        user = client.searchForUsers(row['Person_1'])[0].uid
        messagList.append(user)
        time.sleep(2)
        user2 = client.searchForUsers(row['Person_2'])[0].uid
        messagList.append(user2)
        time.sleep(2)
        user3 = client.searchForUsers(row['Person_3'])[0].uid
        messagList.append(user3)
        time.sleep(2)
        if row['Person_4'] is not None:
            user4 = client.searchForUsers(row['Person_4'])[0].uid
            messagList.append(user4)
        client.createGroup("Hi, welcome to your first grouping!! Use this group to either virtually hangout, play a game and get to know each other. You don't have to use this group but it'd be cool for y'all to catch up and get to know each other if you don't already", messagList)
        time.sleep(10)

In [ ]:
display_groups['Group'] = display_groups.apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
display_groups

The below code snippet links this notebook to your Airtable Base. For more information on how to do that contact Vaibhav Gattani at vaibg@berkeley.edu

In [ ]:
display_groups_filtered = display_groups[['Group']]
records = display_groups_filtered.to_dict('records')
at = airtable.Airtable('BASE ID', 'API KEY') #EDIT HERE

In [ ]:
#Create all records
for r in records:
    at.create(table_name, r)

In [ ]:
#Fetch all records
our_table = at.get(table_name)
filtered_table = [record['fields'] for record in our_table['records']]
fetch_groups = pd.DataFrame(filtered_table)
fetch_groups['createdTime'] = [record['createdTime'] for record in our_table['records']]
fetch_groups = fetch_groups.sort_values("createdTime", ascending=False)
fetch_groups = fetch_groups.drop_duplicates(subset=["Group"])

In [ ]:
merged_groups = fetch_groups.merge(display_groups, on='Group')
merged_groups

In [ ]:
#Update weights for each group
for index, row in merged_groups.iterrows():
    
    if row["Attended"] is True:
        memberNames[row["Person_1"]][row["Person_2"]] = POPA
        memberNames[row["Person_1"]][row["Person_3"]] = POPA
        memberNames[row["Person_2"]][row["Person_1"]] = POPA
        memberNames[row["Person_2"]][row["Person_3"]] = POPA
        memberNames[row["Person_3"]][row["Person_2"]] = POPA
        memberNames[row["Person_3"]][row["Person_1"]] = POPA
        if row["Person_4"] is not None:
            memberNames[row["Person_1"]][row["Person_4"]] = POPA
            memberNames[row["Person_2"]][row["Person_4"]] = POPA
            memberNames[row["Person_3"]][row["Person_4"]] = POPA
            memberNames[row["Person_4"]][row["Person_1"]] = POPA
            memberNames[row["Person_4"]][row["Person_2"]] = POPA
            memberNames[row["Person_4"]][row["Person_3"]] = POPA
    
    

Now that we've updated our weights, we can delete all records in our Airtable Base and create new ones of the next week's pairings by running the matching algorithm again!

In [ ]:
#Delete all records
our_table = at.get(table_name)
for r in our_table['records']:
    at.delete(table_name, r['id'])

In [ ]:
weightings = pd.DataFrame(memberNames)
weightings.to_csv('weightings.csv')